# Ноутбук для решения задач для урока М4 Basic-Advansic prompting

In [1]:
!pip install openai langchain langchain-openai langchain-experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.5/449.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from getpass import getpass

## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [ ]:
import os
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇

In [3]:
from langchain_openai import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ, полученный в боте курса"
course_api_key = getpass(prompt='Введите ваш API ключ, полученный в боте курса')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите ваш API ключ, полученный в боте курса··········


## Задание 4.3.6 Судоку 🔢



In [4]:
# если давать условие на английском, модель работает лучше
sudoku_puzzle = "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"
problem_description = f"""
{sudoku_puzzle}

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.
""".strip()
print(problem_description)

3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1

- This is a 4x4 Sudoku puzzle.
- The * represents a cell to be filled.
- The | character separates rows.
- At each step, replace one or more * with digits 1-4.
- There must be no duplicate digits in any row, column or 2x2 subgrid.
- Keep the known digits from previous valid thoughts in place.
- Each thought can be a partial or the final solution.


In [ ]:
sudoku_puzzle = "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"
problem_description = f"""
{sudoku_puzzle}
- Это головоломка Судоку размером 4x4.
- Символ * обозначает ячейку, которую тебе нужно заполнить.
- Символ | разделяет строки.
- На каждом шаге замени одну или несколько * цифрами от 1 до 4.
- Напиши текущее состояние пазла с добавленными цифрами
- Ни в одной строке, столбце или 2x2 подсетке не должно быть одинаковых цифр.
- Сохраняй полученные цифры из предыдущих успешных мыслей на своих местах.
- Каждая мысль может быть промежуточным или окончательным решением.
""".strip()
print(problem_description)

3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1
- Это головоломка Судоку размером 4x4.
- Символ * обозначает ячейку, которую тебе нужно заполнить.
- Символ | разделяет строки.
- На каждом шаге замени одну или несколько * цифрами от 1 до 4.
- Напиши текущее состояние пазла с добавленными цифрами
- Ни в одной строке, столбце или 2x2 подсетке не должно быть одинаковых цифр.
- Сохраняй полученные цифры из предыдущих успешных мыслей на своих местах.
- Каждая мысль может быть промежуточным или окончательным решением.


In [8]:
from typing import Tuple
from langchain_experimental.tot.checker import ToTChecker
from langchain_experimental.tot.thought import ThoughtValidity


def _parse_grid(grid_str: str):
    if not grid_str.strip():
        raise ValueError("Empty grid string")
    clean_str = grid_str.replace(" ", "")
    rows = clean_str.split('|')
    return [row.split(',') for row in rows]


class MyChecker(ToTChecker):
    def evaluate(self, problem_description: str, thoughts: Tuple[str, ...] = ()) -> ThoughtValidity:
        if not thoughts:
            return ThoughtValidity.INVALID

        current_state = thoughts[-1].strip()
        if not current_state:
            return ThoughtValidity.INVALID

        # === Попытка извлечь original_puzzle ===
        original_puzzle = None
        lines = [line.strip() for line in problem_description.strip().split('\n') if line.strip()]
        if lines:
            original_puzzle = lines[0]

        try:
            current_grid = _parse_grid(current_state)
        except Exception:
            return ThoughtValidity.INVALID

        if len(current_grid) != 4 or any(len(row) != 4 for row in current_grid):
            return ThoughtValidity.INVALID

        # === Проверка символов ===
        for row in current_grid:
            for cell in row:
                if cell != '*' and (not cell.isdigit() or int(cell) not in {1, 2, 3, 4}):
                    return ThoughtValidity.INVALID

        # === Проверка фиксированных ячеек (только если original_puzzle задан) ===
        if original_puzzle is not None:
            try:
                original_grid = _parse_grid(original_puzzle)
                if len(original_grid) != 4 or any(len(row) != 4 for row in original_grid):
                    return ThoughtValidity.INVALID
                for i in range(4):
                    for j in range(4):
                        orig_cell = original_grid[i][j]
                        curr_cell = current_grid[i][j]
                        if orig_cell != '*' and orig_cell != curr_cell:
                            return ThoughtValidity.INVALID
            except Exception:
                return ThoughtValidity.INVALID

        # === Проверка правил Судоку ===
        def _is_valid_group(group):
            filled = [x for x in group if x != '*']
            return len(filled) == len(set(filled))

        # Строки
        for row in current_grid:
            if not _is_valid_group(row):
                return ThoughtValidity.INVALID

        # Столбцы
        for col in range(4):
            column = [current_grid[row][col] for row in range(4)]
            if not _is_valid_group(column):
                return ThoughtValidity.INVALID

        # Блоки 2x2
        for block_row in range(2):
            for block_col in range(2):
                block = []
                for i in range(2):
                    for j in range(2):
                        block.append(current_grid[block_row * 2 + i][block_col * 2 + j])
                if not _is_valid_group(block):
                    return ThoughtValidity.INVALID

        # === Решение завершено? ===
        is_complete = all(cell != '*' for row in current_grid for cell in row)
        if is_complete:
            return ThoughtValidity.VALID_FINAL
        else:
            return ThoughtValidity.VALID_INTERMEDIATE

Проверьте, правильно ли работает ваш чекер на тестах. Можете также написать свои тесты.

In [9]:
checker = MyChecker()
assert (
    checker.evaluate("", ("3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1",))
    == ThoughtValidity.VALID_INTERMEDIATE
)
assert (
    checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1",))
    == ThoughtValidity.VALID_FINAL
)
assert (
    checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1",))
    == ThoughtValidity.VALID_INTERMEDIATE
)
assert (
    checker.evaluate("", ("3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1",))
    == ThoughtValidity.INVALID
)

In [10]:
# запустим tot, если хотите (будут тратиться токены)
from langchain_experimental.tot.base import ToTChain

tot_chain = ToTChain(
    llm=llm, checker=MyChecker(), k=30, c=2, verbose=True, verbose_llm=False
)
tot_chain.invoke({'problem_description': problem_description})



> Entering new ToTChain chain...
Starting the ToT solve procedure.


/usr/local/lib/python3.12/dist-packages/langchain_experimental/tot/thought_generation.py:83: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  new_thoughts = self.predict_and_parse(


        Thought: 3,4,*,2|1,2,3,*|*,1,*,3|4,*,*,1


/usr/local/lib/python3.12/dist-packages/langchain_experimental/tot/thought_generation.py:83: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  new_thoughts = self.predict_and_parse(


    Thought: 3, *, *, 2 | 1, 2, 3, 4 | *, 1, *, 3 | 4, *, *, 1
Thought: 3,*,*,2|1,*,3,4|*,1,*,3|4,*,*,1


...

/usr/local/lib/python3.12/dist-packages/langchain_experimental/tot/thought_generation.py:83: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  new_thoughts = self.predict_and_parse(


                    Thought: 3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1


/usr/local/lib/python3.12/dist-packages/langchain_experimental/tot/thought_generation.py:83: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  new_thoughts = self.predict_and_parse(


                        Thought: 3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1

> Finished chain.


{'problem_description': '3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1\n\n- This is a 4x4 Sudoku puzzle.\n- The * represents a cell to be filled.\n- The | character separates rows.\n- At each step, replace one or more * with digits 1-4.\n- There must be no duplicate digits in any row, column or 2x2 subgrid.\n- Keep the known digits from previous valid thoughts in place.\n- Each thought can be a partial or the final solution.',
 'response': '3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1'}

## Задание 4.3.7 🥊PAL против математических задачек🧑‍🎓






Подгрузим датасет

In [11]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("https://stepik.org/media/attachments/lesson/1282589/math_pal.csv")

In [51]:
from langchain_experimental.pal_chain.base import PALChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain


In [59]:
from langchain_openai import ChatOpenAI

course_api_key = getpass(prompt="Введите ваш ключ, полученный в боте курса")

llm = ChatOpenAI(
    api_key=course_api_key,
    model='gpt-4o-mini',
    base_url="https://aleron-llm.neuraldeep.tech/"
)

# Создание PALChain с модифицированным промптом ===
from langchain_experimental.pal_chain.base import PALChain

# Создаём цепочку с дефолтным математическим промптом
pal_chain = PALChain.from_math_prompt(llm, allow_dangerous_code=True, verbose=True)

# Модифицируем промпт: запрещаем Markdown и требуем только код
original_template = pal_chain.llm_chain.prompt.template
strict_instructions = (
    "Write ONLY valid Python code. Do NOT use markdown code blocks (```python ... ``` or ```).\n"
    "Use pi = 3.14159 if needed. Print ONLY the final number. No explanations, no comments.\n\n"
)
pal_chain.llm_chain.prompt.template = strict_instructions + original_template

In [ ]:
df

Сделайте предсказания. Чтобы избежать импортов библиотеки math, можо подавать в инпут значение числа "пи" 3.14159.
Помните, что мы хотим получить число, а ЛЛМка будет выдавать строку.

In [63]:
answers = []
for task in tqdm(df['task'], desc="Решение задач"):
    try:
        # Добавляем уточнение про pi (на всякий случай)
        question = f"{task}. Use pi = 3.14159 if needed."
        result = pal_chain.invoke({"question": question})
        ans = result["result"]

        # Приведение к числу: float → int, если целое
        if isinstance(ans, float):
            if ans.is_integer():
                ans = int(ans)
            else:
                # Округляем до 6 знаков, чтобы избежать 3.1415900000000002
                ans = round(ans, 6)
        elif isinstance(ans, str):
            # Иногда возвращается строка вроде '700' или '106.814'
            try:
                f_val = float(ans.strip())
                if f_val.is_integer():
                    ans = int(f_val)
                else:
                    ans = round(f_val, 6)
            except ValueError:
                ans = None
        answers.append(ans)
    except Exception as e:
        print(f"\nОшибка при решении задачи: {task[:60]}... → {e}")
        answers.append(None)  # продолжаем обработку

Решение задач:   0%|          | 0/10 [00:00<?, ?it/s]



> Entering new PALChain chain...
def solution():
    length = 1.5
    width = 2
    perimeter = 2 * (length + width) * 100
    return perimeter


Решение задач:  10%|█         | 1/10 [00:01<00:16,  1.87s/it]


> Finished chain.


> Entering new PALChain chain...
def solution():
    pi = 3.14159
    radius1 = 5
    radius2 = 3
    area1 = pi * (radius1 ** 2)
    area2 = pi * (radius2 ** 2)
    total_area = area1 + area2
    return total_area


Решение задач:  20%|██        | 2/10 [00:04<00:16,  2.03s/it]

...


> Finished chain.


> Entering new PALChain chain...
def solution():
    a = 30
    b = 40
    c = int('110010', 2)
    s = (a + b + c) / 2
    area = (s * (s - a) * (s - b) * (s - c)) ** 0.5
    return area


Решение задач:  90%|█████████ | 9/10 [00:17<00:02,  2.02s/it]


> Finished chain.


> Entering new PALChain chain...
def solution():
    pi = 3.14159
    radius1 = 8
    height1 = 20
    volume1 = pi * (radius1 ** 2) * height1
    radius2 = 12
    height2 = 25
    volume2 = pi * (radius2 ** 2) * height2
    total_volume = volume1 + volume2
    total_volume_liters = total_volume / 1000
    result = total_volume_liters
    return result


Решение задач: 100%|██████████| 10/10 [00:21<00:00,  2.13s/it]


> Finished chain.


In [64]:
answers

[700, 106.81406, 3, 901, 9.42477, 2094.393333, 7, 1111101, 600, 15.330959]

In [65]:
df['answer'] = answers # запишем предсказания в датафрейм

In [67]:
df[['task', 'answer']].to_csv("5.6.7_solution.csv", index=False) # сохраним решение в csv

In [69]:
df['answer'][0]

np.float64(700.0)

In [68]:
pd.read_csv('/content/5.6.7_solution.csv')

,task,answer
0,Периметр прямоугольника со сторонами 1.5 м и 2...,7.000000e+02
1,Найди сумму площадей круга с радиусом 5 см и к...,1.068141e+02
2,Переведи числа 20 и 32 из десятичной в двоичну...,3.000000e+00
3,Переведи числа из двоичной системы счисления 1...,9.010000e+02
4,Найди объем цилиндра с радиусом 10 см и высото...,9.424770e+00
5,"Найди длину дуги окружности с радиусом 20 м, о...",2.094393e+03
6,Переведи десятичное число 1524 в двоичную сист...,7.000000e+00
7,"Найти объем куба со стороной 5 см, затем перев...",1.111101e+06
8,Стороны треугольника соответственно равны 30 с...,6.000000e+02
9,Найди объемы двух цилиндров с радиусами 8 см и...,1.533096e+01
